In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


# Building out the CNN using VGG16 architecture

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout 
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [175]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],  # Adjust brightness
    channel_shift_range=60.0,  # Randomly shift pixel values in each image channel
    fill_mode='nearest' 
)

train_generator = train_datagen.flow_from_directory(
    directory= r'data\train',  # Directory where labeled data is stored
    target_size=(150, 150),  # Image size for VGG16
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    directory= r'data\train',  # Ensure this is pointing to the same directory (not 'tester')
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Validation set with one-hot encoded labels
    subset='validation'
)

Found 1863 images belonging to 11 classes.
Found 460 images belonging to 11 classes.


In [177]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
for layer in base.layers:
    layer.trainable = False
model = Sequential([
    base,
    Flatten(),  # Flatten the feature maps from the convolutional layers
    #Dropout(0.5),
    Dense(128, activation='relu'),  # Add a fully connected layer
    #Dropout(0.5),  # Dropout to prevent overfitting
    Dense(train_generator.num_classes, activation='softmax')  # Output layer for cloud classification
])

In [179]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('cloud_classification_VGG.keras', monitor='val_loss', save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=5,              # Number of epochs with no improvement after which training will be stopped
    mode='min',              # Mode: 'min' for loss, 'max' for accuracy
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=30, # You can increase the epochs if needed
    callbacks=[early_stopping, model_checkpoint] # Include the early stopping callback

)



Epoch 1/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 52s 882ms/step - accuracy: 0.1664 - loss: 2.3616 - val_accuracy: 0.2902 - val_loss: 2.0223
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4062 - loss: 1.9802 - val_accuracy: 0.3333 - val_loss: 2.0169
Epoch 3/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 51s 875ms/step - accuracy: 0.3484 - loss: 1.9807 - val_accuracy: 0.2924 - val_loss: 1.9420
Epoch 4/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2812 - loss: 1.9998 - val_accuracy: 0.2500 - val_loss: 1.9629
Epoch 5/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 51s 872ms/step - accuracy: 0.3672 - loss: 1.8562 - val_accuracy: 0.3281 - val_loss: 1.8970
Epoch 6/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3125 - loss: 1.9707 - val_accuracy: 0.4167 - val_loss: 1.8722
Epoch 7/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 51s 883ms/step - accuracy: 0.3820 - loss: 1.8148 - val_accuracy: 0.3170 - val_loss: 1.8813
Epoch 8/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4062 - loss: 1.8108 - val_accuracy: 0.16

In [180]:
#model.save('cloud_classification_VGG.keras')

In [183]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model

In [185]:
model = load_model('cloud_classification_VGG.keras')
def classify_cloud(img_path, model):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))  # Resizing the image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess for VGG

    # Feed the preprocessed image to the model
    predictions = model.predict(img_array)

    # Interpret the result
    predicted_class = np.argmax(predictions, axis=1)  # Get the index of the highest probability

    return predicted_class

# IMAGE INPUT 

In [219]:
#your image of cloud goes here in image_path variable***
image_path = r'C:\Users\Jaylen\Desktop\cloud1.jpg'
classification = {
    0: "Altocumulus",
    1: "Altostratus",
    2: "Cumulonimbus",
    3: "Cirrocumulus",
    4: "Cirrus",
    5: "Cirrostratus",
    6: "Contrail",
    7: "Cumulus",
    8: "Nimbostratus",
    9: "Stratocumulus",
    10: "Stratus"
}

In [221]:
result = classify_cloud(image_path, model)
print(f"Predicted cloud type: {classification[result[0]]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted cloud type: Stratocumulus
